In [9]:
%%time
import sys
import os 
module_path = os.path.abspath(os.path.join('..')) 
if module_path not in sys.path: 
    sys.path.append(module_path)
import tools.Sample_Tools as smpl
import tools.Factor_Cacher as cacher
import tools.Pretreat_Tools as pretreat
from base.JuUnits import excute_for_multidates

import pandas as pd
import numpy as np

%load_ext autoreload
%autoreload 2
%aimport tools.Factor_Cacher


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Wall time: 3.62 ms


In [3]:
%%time
# factors = cacher.load_caches_adv(['alpha11','alpha24','alpha38','alpha41'],'2021-11-18','2021-11-24')
# codes = factors.index.get_level_values(1).unique()
# dates = factors.index.get_level_values(0).unique()
# data = smpl.get_data(codes.to_list(),dates.min().strftime('%Y-%m-%d'),dates.max().strftime('%Y-%m-%d')).data
# mv_industry = smpl.add_marketvalue_industry(data).drop(columns=data.columns.to_list())
# industy = mv_industry['industry']
# cap = mv_industry['totalCapital']
# adj_weight = np.sqrt(cap)/np.sqrt(cap).sum()
# # adj_weight_mat = pd.DataFrame(np.diag(adj_weight),index=adj_weight.index, columns=adj_weight.index)
# ret_forward = smpl.get_forward_return(data,'close')
# ret_forward.name = 'ret_forward'

# factors = factors.assign(country=1.0, industry = industy)
# factors = pd.get_dummies(factors,'industry')
# del factors['country']
# factors

Wall time: 4.3 s


alpha11  alpha24   alpha38   alpha41  industry_PCB  \
date       code                                                          
2021-11-18 000001  1.081522    -0.45 -0.071362  0.044701             0   
           000002  0.152750    -1.45 -0.137740  0.040979             0   
           000063  0.364765    -0.28 -0.349654  0.029828             0   
           000066  0.318474    -0.73 -0.032221  0.009315             0   
           000069  0.631213    -0.28 -0.057293  0.012834             0   
...                     ...      ...       ...       ...           ...   
2021-11-24 603882  0.752749   -17.21 -0.282632  0.957443             0   
           603899  0.716829    -0.00 -0.031443  0.082944             0   
           603986  1.171785   -13.87 -0.050580  0.176878             0   
           603993  0.160092    -0.25 -0.611960 -0.023814             0   
           605499  0.524685   -15.55 -0.660235  0.067494             0   

                   industry_中药  industry_乳制品  industry_云软件服务  \
date       code                                                
2021-11-18 000001            0             0               0   
           000002            0             0               0   
           000063            0             0               0   
           000066            0             0               0   
           000069            0             0               0   
...                        ...           ...             ...   
2021-11-24 603882            0             0               0   
           603899            0             0               0   
           603986            0             0               0   
           603993            0             0               0   
           605499            0             0               0   

                   industry_产联网信息服务  industry_住宅开发  ...  industry_铜  \
date       code                                     ...               
2021-11-18 000001                 0              0  ...           0   
           000002                 0              1  ...           0   
           000063                 0              0  ...           0   
           000066                 0              0  ...           0   
           000069                 0              1  ...           0   
...                             ...            ...  ...         ...   
2021-11-24 603882                 0              0  ...           0   
           603899                 0              0  ...           0   
           603986                 0              0  ...           0   
           603993                 0              0  ...           0   
           605499                 0              0  ...           0   

                   industry_铝  industry_锂  industry_锂电池  industry_防水材料  \
date       code                                                          
2021-11-18 000001           0           0             0              0   
           000002           0           0             0              0   
           000063           0           0             0              0   
           000066           0           0             0              0   
           000069           0           0             0              0   
...                       ...         ...           ...            ...   
2021-11-24 603882           0           0             0              0   
           603899           0           0             0              0   
           603986           0           0             0              0   
           603993           0           0             0              0   
           605499           0           0             0              0   

                   industry_集成电路设计  industry_面板  industry_风力发电  industry_风电整机  \
date       code                                                                 
2021-11-18 000001                0            0              0              0   
           000002                0            0              0              0   
           00

In [8]:
section = factors.loc['2021-11-24']
k=section.shape[1]
diag_r = np.diag(np.ones(k))
location = k-1
r=np.delete(diag_r,location,axis=1)

r


array([[1., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [ ]:
def calc_wgt(stocks,date):
        #计算个股权重矩阵
        #个股权重为流通市值平方和占比
        _float_mv_temp = np.sqrt(_float_mv.T.loc[stocks,date])
        _sum = _float_mv_temp.sum()
        _wgt = _float_mv_temp/_sum
        _V = pd.DataFrame(np.diag(_wgt),columns = stocks,index = stocks)
        return _V
    V = calc_wgt(stocks,date)#个股权重矩阵，权重为流通市值平方占比
    
    def calc_R(ind,factor):
        #计算约束矩阵防止共线性问题
        #获取申万28个行业列表
        _sw_ind = list(ind.drop_duplicates().values)
        
        ind_float_mv = pd.DataFrame(index = stocks,columns = ['industry','float_mv'])
        ind_float_mv['industry'] = ind
        ind_float_mv['float_mv'] = _float_mv.T.loc[stocks,date]
        _sum_float_mv = ind_float_mv['float_mv'].sum()
        
        _industry_wgt = pd.Series(index = _sw_ind)
        
        for i in _sw_ind:
            #计算行业流通市值占比
            _industry_wgt[i] = ind_float_mv[ind_float_mv['industry']==i]['float_mv'].sum()/_sum_float_mv
        #构建约束矩阵R，R为K*K-1矩阵，K为因子数量
        #将最后一个行业因子表示为其他行业的线性形式
        #移除最后一个行业所在的列，并对其所在的行进行权重处理
        k = factor.shape[1] #总因子数量
        p = len(_industry_wgt) #行业因子数量
        q = k-1-p #风格因子数量
        
        diag_R = np.diag(np.ones(k))
        location = len(_industry_wgt)
        R = np.delete(diag_R,location,axis = 1)
        adj_industry_weights = -_industry_wgt.div(_industry_wgt[-1]).iloc[:-1]
        R[location,1:p] = adj_industry_weights
        
        return R


In [5]:
# adj_weight
cap

date        code  
2021-11-18  000001    3.454254e+11
            000002    2.217825e+11
            000063    1.504682e+11
            000066    3.975648e+10
            000069    5.085112e+10
                          ...     
2021-11-24  603882    5.089164e+10
            603899    5.458507e+10
            603986    1.040930e+11
            603993    1.282995e+11
            605499    7.714193e+10
Name: totalCapital, Length: 1239, dtype: float64

In [57]:
# import Analysis_Funs as af
dat='2021-11-18'
f = af.get_LR_params_fast(factors.loc[dat,:].values,ret_forward[dat].values)
f
# factors.shape


array([-6.46650584e-03, -1.75750531e-04,  2.26191833e-01, -1.51680769e-02,
       -2.77526332e-02,  4.90412624e-02,  2.80169458e-02,  3.99594053e-02,
        6.22880280e-02,  8.39990202e-02,  2.13886050e-02,  1.19790836e-01,
        6.36815021e-02,  4.55100654e-01,  1.00350810e-01,  1.17391245e-01,
        1.14322960e-01,  2.12301140e-01,  1.64441379e-02,  8.78179646e-02,
        9.32841778e-02,  3.02392768e-02,  1.05842160e-01, -2.65043394e-01,
        3.05797947e-02,  8.75976409e-02,  1.96671778e-01,  2.27805119e-02,
        2.39188849e-02, -5.56836596e-03,  3.59830702e-03,  1.17040205e-01,
       -2.57177335e-01,  1.10796742e-01,  4.69772759e-02,  1.96436340e-02,
        2.11120839e-01,  5.13046357e-02, -1.68914983e-01,  9.63181777e-02,
        5.84696095e-02,  3.72536812e-02,  2.67597330e-02,  3.45860265e-01,
        4.57719039e-02,  7.02498816e-02,  6.62648132e-02,  4.33388916e-02,
        3.19517852e-01,  1.04566079e-01,  3.88351415e-01,  4.64080943e-02,
        3.23087525e-02,  

In [58]:
ret_forward[dat].values

array([ 1.96629213e-02,  4.24305919e-02,  5.34255185e-03,  1.69992609e-02,
        2.41935484e-02,  9.52380952e-03,  1.45348837e-03,  1.76817289e-02,
        3.44827586e-02, -3.13301054e-03,  4.68140442e-02,  3.32778702e-03,
        2.22222222e-04,  3.30719247e-02, -3.83171124e-05,  8.30564784e-03,
        4.47552448e-03, -1.67901406e-02, -1.91021968e-03, -1.03189493e-02,
        8.17995910e-03, -1.34770889e-03,  6.23229462e-02,  1.38696255e-02,
        3.01027900e-02,  9.61538462e-03,  1.90777490e-02, -1.34788190e-02,
        3.98936170e-03, -8.45899228e-03,  3.80331432e-03,  1.07019201e-02,
        6.37707948e-02,  9.99666778e-03,  1.37551582e-03,  5.45627829e-02,
        2.46305419e-03,  5.54785021e-03,  2.25080386e-03,  1.43415906e-02,
       -1.00559719e-02,  1.07573150e-02,  1.42314991e-02,  1.00105374e-01,
        1.42014420e-02,  1.67890871e-02, -2.46575342e-02, -2.30109204e-02,
        1.17083555e-02,  6.37055388e-03,  4.80065094e-02,  5.77154309e-02,
        3.05810398e-03,  

In [68]:
from QUANTAXIS.QAUtil import (
    DATABASE)
# a = smpl.get_blocks_view('concept',collections=DATABASE.stock_block_em)
a['农业种植']

['600359',
 '000568',
 '600084',
 '600543',
 '300162',
 '600313',
 '002582',
 '300858',
 '000716',
 '300992',
 '601118',
 '601996',
 '000850',
 '603336',
 '002042',
 '000623',
 '002999',
 '600506',
 '300543',
 '603789',
 '300829',
 '000726',
 '600057',
 '300237',
 '600540',
 '600251',
 '002556',
 '002470',
 '600811',
 '300586',
 '000937',
 '300138',
 '300175',
 '600731',
 '300158',
 '300495',
 '002566',
 '002539',
 '002772',
 '000667',
 '301035',
 '600108',
 '600737',
 '600598',
 '600371',
 '002717',
 '002200',
 '300087',
 '600354',
 '002588',
 '002041',
 '000998',
 '601952',
 '002215',
 '300908',
 '300511',
 '002385',
 '000902',
 '002538',
 '002326',
 '002312',
 '000713',
 '300972',
 '300189',
 '002758',
 '003042',
 '300970',
 '300387']

In [3]:
from QUANTAXIS.QAUtil import (
    DATABASE)
import QUANTAXIS as QA
import numpy as np
import pandas as pd
pd.get_dummies

# a = pd.DataFrame(np.array([[1,2,3,4,5],[2,3,1,1,2]]).T,columns=['A','B'])
a = pd.DataFrame(np.array([[1,2,3,4,5],[0.1,0.2,0.3,0.4,0.5],['a','b','a','a','c']]).T,columns=['AT','AT2','BT'])
print(a)
c = pd.get_dummies(a,columns=['BT'])
print(c)

  AT  AT2 BT
0  1  0.1  a
1  2  0.2  b
2  3  0.3  a
3  4  0.4  a
4  5  0.5  c
  AT  AT2  BT_a  BT_b  BT_c
0  1  0.1     1     0     0
1  2  0.2     0     1     0
2  3  0.3     1     0     0
3  4  0.4     1     0     0
4  5  0.5     0     0     1


In [4]:
sum_classic = c['AT'].astype('int32').T.dot(c.iloc[:,2:5])
print(sum_classic)
x = (c.iloc[:,2:5] * sum_classic).sum(axis=1)
c['AT2'] = c['AT2'].astype('float32')/x
print(c)

BT_a    8
BT_b    2
BT_c    5
Name: AT, dtype: int32
  AT     AT2  BT_a  BT_b  BT_c
0  1  0.0125     1     0     0
1  2  0.1000     0     1     0
2  3  0.0375     1     0     0
3  4  0.0500     1     0     0
4  5  0.1000     0     0     1
